In [ ]:
from ftx_client import FtxClient
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from supertrend import supertrend_analysis, get_supertrend_signals, calculate_sma_200
from config import API_KEY, API_SECRET
import ipympl
from pprint import pprint
import numpy as np
import time
from telegram_api_manager import TelegramAPIManager
import json
import os
plt.ioff()
FIGURE_PATH = "figures"

In [ ]:
perpetuals = [
    "ETH-PERP", 
    "MATIC-PERP", 
    "SOL-PERP", 
    "DOGE-PERP", 
    "ADA-PERP", 
    "RUNE-PERP", 
    "CHZ-PERP", 
    "SUSHI-PERP", 
]

In [ ]:
trades = {}
for perp in perpetuals:
    trades[perp] = []

tapi = TelegramAPIManager(group=False)

while(True):
    # Open new FTX Session
    ftx = FtxClient(api_key=API_KEY, api_secret=API_SECRET)

    for perp in perpetuals:
        df = ftx.get_historical_market_data(perp, interval="1h", start_time="40 days ago")

        # Perform supertrend analysis
        st, upt, dt = supertrend_analysis(df.high, df.low, df.close, lookback=10, multiplier=3)
        long_trigger, short_trigger, st_signal = get_supertrend_signals(df.close, st)

        # Check 200 SMA for confirmation
        sma200 = calculate_sma_200(df)
        
        # Set precision for orders
        precision = len(str(df.close[0]).split(".")[1])

        # Plot and save results
        fig, ax = plt.subplots()
        ax.plot(df.index, df.close, ".-", color="tab:blue")
        ax.plot(df.index, sma200, label="200 SMA", color="tab:orange")
        ax.plot(df.index[1:], st, color="tab:gray")
        
        ax.plot(df.index[1:], long_trigger, marker='^', color='tab:green', markersize=8, linewidth=0, label='Short')
        ax.plot(df.index[1:], short_trigger, marker='v', color='tab:red', markersize=8, linewidth=0, label='Long')
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%b-%d %H:%m'))
        fig.autofmt_xdate()
        ax.set_title(perp)
        plt.close(fig)
        fig.savefig(os.path.join(FIGURE_PATH, f"{perp}.jpg"), dpi=300)

        # Check last element of signal array
        last_signal, idx = st_signal[-1], -1
        # for idx, last_signal in enumerate(st_signal):
        if last_signal != 0:
            if last_signal == 1:
                new_position = {"perp": perp, "position": "long", "entry": df.close[idx], "stop_loss": round(st[idx], precision), "10pctprofit": df.close[idx] + df.close[idx]*0.1}
                if df.close[-1] > sma200[-1]:
                    new_position["sma200"] = "over"
            elif last_signal == -1:
                new_position = {"perp": perp, "position": "short", "entry": df.close[idx], "stop_loss": round(st[idx], precision), "10pctprofit": df.close[idx] - df.close[idx]*0.1}
                if df.close[-1] < sma200[-1]:
                    new_position["sma200"] = "under"

            trades[perp].append(new_position)
            tapi.send_photo(os.path.join(FIGURE_PATH, f"{perp}.jpg"), caption=json.dumps(new_position, indent=2, default=str))
        else:
            pass
    print("No signals found, sleeping for 60 mins")
    time.sleep(60*60)

In [ ]:
def calculate_pnl(perpetuals, trades, leverage=1):
    pnls = []
    for perp in perpetuals:
        if len(trades[perp]) >= 2:
            for idx in range(len(trades[perp])):
                if idx + 1 < len(trades[perp]):
                    if trades[perp][idx]["position"] == "short":
                        pnl = (trades[perp][idx]["entry"] - trades[perp][idx+1]["entry"]) / trades[perp][idx+1]["entry"] * 100 
                    elif trades[perp][idx]["position"] == "long":
                        pnl = (trades[perp][idx+1]["entry"] - trades[perp][idx]["entry"]) / trades[perp][idx]["entry"] * 100
                    pnl *= leverage
                    print(f"{perp} {round(pnl, 1)}%")
                    pnls.append(pnl)

    print(f"Average pnl = {sum(pnls) / len(pnls)}")

In [ ]:
calculate_pnl(perpetuals, trades, 10)

In [ ]:
print(json.dumps(new_position, indent=2, default=str))